Consentono di integrare LLM con altri tool specifici per determinati task (ad es. calcolatrice, motore di ricerca, eseguire codice, ecc).

Langchain dispone di Tools che consentono di integrare LLMs consentendo l'accesso da parte di questi a :
- motori di ricerca
- database
- API
- ecc

Iniziamo da un search tool: Duckduckgo e Wikipedia

In [2]:
pip install -q duckduckgo-search

Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

output = search.invoke("When was Roberto Baggio born?")
output

"Roberto Baggio was born on February 18, 1967, in Caldogno, Italy. He showed early talent in soccer and joined the Vicenza youth team at the age of 13. Baggio's Iconic Jersey Number. Throughout his career, Baggio was known for wearing the number 10 jersey, which became synonymous with his name. This number is often associated with creative and ... Roberto Baggio was born on February 18, 1967, in Caldogno, a small Italian town. The sixth of eight siblings, Baggio's passion for football was evident from a young age. Driven by a deep love for the game, he often played with a fervent zest that set him apart from his peers. Former Italian professional soccer player Roberto Baggio was born in Caldogno, Italy, on 18 February 1967, making Aquarius his zodiac sign. He played in various attacking positions , and is still today regarded as one of the best soccer players of all time; Roberto was voted #4 in the internet poll FIFA Player of the Century in 1999, and was ... Roberto Baggio was born i

In [10]:
search.name

'duckduckgo_search'

In [11]:
search.description

'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.'

E' possibile a richiedere un output strutturato come quello offerto dai motori di ricerca con i risultati strutturati con: 
- snippet
- description
- link

In [13]:
from langchain.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()
output = search.run("When was Roberto Baggio born?")
output

"[snippet: Roberto Baggio was born on February 18, 1967, in Caldogno, Italy. He showed early talent in soccer and joined the Vicenza youth team at the age of 13. Baggio's Iconic Jersey Number. Throughout his career, Baggio was known for wearing the number 10 jersey, which became synonymous with his name. This number is often associated with creative and ..., title: 15 Facts About Roberto Baggio, link: https://facts.net/history/people/15-facts-about-roberto-baggio/], [snippet: Roberto Baggio was born on February 18, 1967, in Caldogno, a small Italian town. The sixth of eight siblings, Baggio's passion for football was evident from a young age. Driven by a deep love for the game, he often played with a fervent zest that set him apart from his peers., title: Roberto Baggio: A Legendary Football Icon's ... - The Football Educator, link: https://thefootballeducator.com/roberto-baggio/], [snippet: Former Italian professional soccer player Roberto Baggio was born in Caldogno, Italy, on 18 Feb

Per avere un maggior controllo sull'invocazione al motore di ricerca

In [16]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(region="it-it", max_results=3, safesearch="moderate", )
search = DuckDuckGoSearchResults(api_wrapper=wrapper, source="news")
output = search.run("Qual'è la prossima partita della nazionale di volley maschile")
output

"[snippet: Di seguito il calendario completo, il programma dettagliato, gli orari, il palinsesto tv e streaming delle prossime partite dell'Italia nel torneo di volley maschile alle Olimpiadi di Parigi..., title: Volley, quando gioca l'Italia le prossime partite? Date, orari ..., link: https://www.oasport.it/2024/07/volley-quando-gioca-litalia-le-prossime-partite-date-orari-avversarie-tv/], [snippet: Di seguito il calendario completo, il programma dettagliato, gli orari, il palinsesto tv e streaming delle partite dell'Italia nella Nations League 2024 di volley maschile., title: Calendario Nations League volley maschile 2024: orari e date di tutte ..., link: https://www.oasport.it/2024/05/calendario-nations-league-volley-maschile-2024-orari-e-date-di-tutte-le-partite-dellitalia-dove-vederla-in-tv/], [snippet: L'Italia di Fefè De Giorgi vince la seconda partita nella Volleyball Nations League 2024: batte l'Iran in tre set. Scopri com'è andato il match dell'Italvolley e dove vedere la pro

Per convertire questo output in modo strutturato, utilizzo le regexp

In [21]:
import re
pattern = r'snippet: (.*?), title: (.*?), link: (.*?)\],'

matches = re.findall(pattern, output, re.DOTALL) # DOTALL serve per far si che "." 
                                                 # includa tutti i caratteri compresa \n

for snippet, title, link in matches:
    print(f'Snippet : {snippet}\n Title : {title}\n Link : {link}')
    print('-'* 50)

Snippet : Di seguito il calendario completo, il programma dettagliato, gli orari, il palinsesto tv e streaming delle prossime partite dell'Italia nel torneo di volley maschile alle Olimpiadi di Parigi...
 Title : Volley, quando gioca l'Italia le prossime partite? Date, orari ...
 Link : https://www.oasport.it/2024/07/volley-quando-gioca-litalia-le-prossime-partite-date-orari-avversarie-tv/
--------------------------------------------------
Snippet : Di seguito il calendario completo, il programma dettagliato, gli orari, il palinsesto tv e streaming delle partite dell'Italia nella Nations League 2024 di volley maschile.
 Title : Calendario Nations League volley maschile 2024: orari e date di tutte ...
 Link : https://www.oasport.it/2024/05/calendario-nations-league-volley-maschile-2024-orari-e-date-di-tutte-le-partite-dellitalia-dove-vederla-in-tv/
--------------------------------------------------
Snippet : L'Italia di Fefè De Giorgi vince la seconda partita nella Volleyball Nations Le

Un altro tool molto utile è wikipedia tool

In [22]:
!pip install -q wikipedia

In [27]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500, lang='it')
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.invoke({'query' : 'Baggio'})


"Page: Roberto Baggio\nSummary: Roberto Baggio (Caldogno, 18 febbraio 1967) è un ex calciatore italiano, di ruolo attaccante o centrocampista, vicecampione del mondo con la nazionale italiana nel 1994.\nSoprannominato Raffaello da Gianni Agnelli per l'eleganza dello stile di gioco, e colloquialmente noto come Divin Codino per la caratteristica acconciatura, è ritenuto uno dei migliori giocatori nella storia del calcio. Occupa la 16ª posizione (primo italiano) nella speciale classifica dei migliori "